In [1]:
import numpy as np
import pandas as pd
import torch 
from torch import nn as nn
from matplotlib import pyplot as plt
import time
from datetime import datetime
import tqdm
import networkx as nx

In [2]:
class1_index=np.load('./user_index.npy')
class2_attr=np.load('./NYC_poi.npy')
class2_index=np.load("./poi_index.npy")
one2one_index=np.load('./friend_edge.npy')
one2one_attr=np.load('./friend_attr.npy')
one2two_index=np.load('./checkin_edge.npy')
one2two_attr=np.load('./checkin_attr.npy')

In [3]:
import torch_geometric as pyg
from torch_geometric.data import HeteroData
from torch_geometric.nn import MetaPath2Vec

In [4]:
data=HeteroData()

data['class1'].num_nodes=int(max(class1_index)+1)
data['class1'].y_index=torch.from_numpy(class1_index)
data['class2'].num_nodes=int(max(class2_index)+1)
data['class2'].y=torch.from_numpy(class2_attr)
data['class2'].y_index=torch.from_numpy(class2_index)
data['class1','one2one','class1'].edge_index=torch.from_numpy(one2one_index)
data['class1','one2one','class1'].edge_attr=torch.from_numpy(one2one_attr)
data['class1','one2two','class2'].edge_index=torch.from_numpy(one2two_index)
data['class1','one2two','class2'].edge_attr=torch.from_numpy(one2two_attr)

metapath = [
     ('class1','one2one','class1'),
    ('class1','one2two','class2'),
]

In [5]:
data.edge_index_dict

{('class1',
  'one2one',
  'class1'): tensor([[     19,      19,      19,  ..., 1937706, 2005106, 2027423],
         [     54,    1668,   21596,  ..., 1942412, 2012167, 2034242]]),
 ('class1',
  'one2two',
  'class2'): tensor([[2277773, 1239681,  954627,  ..., 2196640,  259706,  584774],
         [   6764,    6764,    6764,  ...,   86809,   85590,   15689]])}

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MetaPath2Vec(data.edge_index_dict, embedding_dim=32,
                     metapath=metapath, walk_length=20, context_size=7,
                     walks_per_node=5, num_negative_samples=5,
                     sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=6)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

In [8]:
def train(epoch, model, log_steps=100, eval_steps=2000):
    model.train()

    total_loss = 0
    for i, (pos_rw, neg_rw) in enumerate(loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if (i + 1) % log_steps == 0:
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Loss: {total_loss / log_steps:.4f}'))
            total_loss = 0

In [9]:
for epoch in range(1,6):
    train(epoch,model)

IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/cseadmin/whj/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/cseadmin/whj/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/home/cseadmin/whj/anaconda3/lib/python3.8/site-packages/torch_geometric/nn/models/metapath2vec.py", line 176, in _sample
    return self._pos_sample(batch), self._neg_sample(batch)
  File "/home/cseadmin/whj/anaconda3/lib/python3.8/site-packages/torch_geometric/nn/models/metapath2vec.py", line 143, in _pos_sample
    batch = adj.sample(num_neighbors=1, subset=batch).squeeze()
  File "/home/cseadmin/whj/anaconda3/lib/python3.8/site-packages/torch_sparse/sample.py", line 22, in sample
    return col[rand]
IndexError: index 22850 is out of bounds for dimension 0 with size